# 安徽安庆市项目4月`CMAQ-ISAM`
## 使用`M3Tools`生成的`IOAPI`接口的`GRIDMASK.nc`
## 导出ASCII数据

---
*@author: Evan*\
*@date: 2023-05-03*

In [16]:
import xarray as xr
import numpy as np

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [17]:
ds = xr.open_dataset('./output/GRIDMASK.nc')
ds = ds.drop_vars(['MSFX2','HT','DLUSE','LWMASK','PURB']).squeeze()
ds

<xarray.Dataset>
Dimensions:   (VAR: 7, DATE-TIME: 2, ROW: 138, COL: 135)
Dimensions without coordinates: VAR, DATE-TIME, ROW, COL
Data variables:
    TFLAG     (VAR, DATE-TIME) int32 ...
    LAT       (ROW, COL) float32 ...
    LON       (ROW, COL) float32 ...
    Anqing    (ROW, COL) float32 ...
    Hefei     (ROW, COL) float32 ...
    Tongling  (ROW, COL) float32 ...
    Chizhou   (ROW, COL) float32 ...
    Jiujiang  (ROW, COL) float32 ...
Attributes: (12/33)
    IOAPI_VERSION:  $Id:: init3.F 178 2015-03-02 16:35:15Z coats             ...
    EXEC_ID:        mcip                                                     ...
    FTYPE:          1
    CDATE:          2023116
    CTIME:          130610
    WDATE:          2023116
    ...             ...
    VGLVLS:         [1.     0.9979]
    GDNAM:          CN3AH_135X138_CR
    UPNAM:          OUTGM3IO        
    VAR-LIST:       LAT             LON             Anqing          Hefei    ...
    FILEDESC:       US EPA COMMUNITY MULTISCALE AIR QUALITY MODEL            ...
    HISTORY:

In [18]:
df = ds.to_dataframe()
df.drop(['TFLAG','LAT','LON'],axis=1,inplace=True)
df

Anqing  Hefei  Tongling  Chizhou  Jiujiang
VAR DATE-TIME ROW COL                                            
0   0         0   0       0.0    0.0       0.0      0.0       1.0
                  1       0.0    0.0       0.0      0.0       1.0
                  2       0.0    0.0       0.0      0.0       1.0
                  3       0.0    0.0       0.0      0.0       1.0
                  4       0.0    0.0       0.0      0.0       1.0
...                       ...    ...       ...      ...       ...
6   1         137 130     0.0    0.0       0.0      0.0       0.0
                  131     0.0    0.0       0.0      0.0       0.0
                  132     0.0    0.0       0.0      0.0       0.0
                  133     0.0    0.0       0.0      0.0       0.0
                  134     0.0    0.0       0.0      0.0       0.0

[260820 rows x 5 columns]

In [19]:
df = df.iloc[:(135*138),:]
# 删去多余的index
df.reset_index(level=[0,1], drop=True, inplace=True)

# m3mask读取的顺序是先COL后ROW
df = df.swaplevel('ROW','COL')

# 将数据设置为int
df = df.astype(int)
df

,,Anqing,Hefei,Tongling,Chizhou,Jiujiang
COL,ROW,,,,,
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1
...,...,...,...,...,...,...
130,137,0,0,0,0,0
131,137,0,0,0,0,0
132,137,0,0,0,0,0


In [23]:
# fortran是从1开始的，将COL和ROW的数值+1
df = df.rename(index=lambda x: x + 1, level=0)
df = df.rename(index=lambda x: x + 1, level=1)
df

,,Anqing,Hefei,Tongling,Chizhou,Jiujiang
COL,ROW,,,,,
1,1,0,0,0,0,1
2,1,0,0,0,0,1
3,1,0,0,0,0,1
4,1,0,0,0,0,1
5,1,0,0,0,0,1
...,...,...,...,...,...,...
131,138,0,0,0,0,0
132,138,0,0,0,0,0
133,138,0,0,0,0,0


In [24]:
regions = ['Anqing','Hefei','Tongling','Chizhou','Jiujiang']

for region in regions:
    df_region = df[region]
    df_region = df_region[(df_region != 0)]
    df_region.to_csv(f'./output/df_{region}.csv',header=False)
